In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive  sample_data


In [0]:
import os
os.chdir('./drive/My Drive/Signature_Forgery')

In [0]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle 
from tqdm import tqdm 
import numpy as np

from keras.utils import to_categorical
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
TRAIN_DIR = 'data/train_set'
TEST_DIR = 'data/test_set'
IMG_SIZE = 28

In [0]:
def label_img(img):
    image_label = img.split('.')
    name = image_label[0]
    # checking if the signature is forged or genuine
    #                            
    
    if name[4:7] == name[-3:]:
        return 1
  
    elif name[4:7] != name[-3:]:
        return 0

def person_img(img):
    label = img.split('.')
    k = label[0]
    return [k[-3:]]


def create_features(DIR):
    features = []
    labels = []
    for img in tqdm(os.listdir(DIR)):
        label = label_img(img)
        person = person_img(img)
       
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        features.append([img])
        labels.append(np.array(label))
    #shuffle(training_data)
    #np.save('train_data.npy', training_data)
    return features

def create_labels(DIR):
    labels = []
    for img in tqdm(os.listdir(DIR)):
        label = label_img(img)
        labels.append(np.array(label))
    return labels

In [0]:
train_features  = np.array(create_features(TRAIN_DIR))
test_features  = np.array(create_features(TEST_DIR))
train_features_reshaped = train_features.reshape(-1, IMG_SIZE,IMG_SIZE, 1)
test_features_reshaped = test_features.reshape(-1, IMG_SIZE,IMG_SIZE, 1)

train_features_reshaped = train_features_reshaped.astype('float32')
test_features_reshaped = test_features_reshaped.astype('float32')
train_features_reshaped = train_features_reshaped / 255.
test_features_reshaped = test_features_reshaped / 255.

train_labels = np.array(create_labels(TRAIN_DIR))
test_labels = np.array(create_labels(TEST_DIR))


train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

100%|██████████| 121/121 [00:00<00:00, 172564.02it/s]


In [0]:
X_train,X_validate,Y_train,Y_validate = train_test_split(train_features_reshaped, train_labels_one_hot,test_size=0.20,random_state = 42)

In [0]:
X_train.shape

(143, 28, 28, 1)

In [0]:
batch_size = 64
epochs = 50
num_classes = 2

In [0]:
model = Sequential()
model.add(Conv2D(64,kernel_size=(3,3),activation = 'linear',input_shape=(IMG_SIZE,IMG_SIZE,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Conv2D(32,kernel_size=(3,3),activation = 'linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Flatten())
model.add(Dense(1000, activation='linear'))
model.add(LeakyReLU(alpha=0.1))   
model.add(Dense(100, activation='linear'))
model.add(LeakyReLU(alpha=0.1))   
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 32)        18464     
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1568)              0         
__________

In [0]:
#es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', verbose=1)

In [0]:

fit = model.fit(X_train, Y_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_validate,Y_validate))

Train on 143 samples, validate on 36 samples
Epoch 1/50
143/143 [==============================] - 2s 12ms/step - loss: 0.7333 - acc: 0.4406 - val_loss: 0.6954 - val_acc: 0.4444
Epoch 2/50
143/143 [==============================] - 1s 4ms/step - loss: 0.6929 - acc: 0.5175 - val_loss: 0.6953 - val_acc: 0.4444
Epoch 3/50
143/143 [==============================] - 1s 5ms/step - loss: 0.6908 - acc: 0.5175 - val_loss: 0.7033 - val_acc: 0.4444
Epoch 4/50
143/143 [==============================] - 1s 5ms/step - loss: 0.6897 - acc: 0.5175 - val_loss: 0.6998 - val_acc: 0.4444
Epoch 5/50
143/143 [==============================] - 1s 5ms/step - loss: 0.6880 - acc: 0.5315 - val_loss: 0.6800 - val_acc: 0.5556
Epoch 6/50
143/143 [==============================] - 1s 5ms/step - loss: 0.6918 - acc: 0.4825 - val_loss: 0.6862 - val_acc: 0.6111
Epoch 7/50
143/143 [==============================] - 1s 4ms/step - loss: 0.6767 - acc: 0.6503 - val_loss: 0.6896 - val_acc: 0.5556
Epoch 8/50
143/143 [==========

In [0]:

score, acc = model.evaluate(test_features_reshaped, test_labels_one_hot,
                            batch_size=batch_size)

121/121 [==============================] - 0s 691us/step


In [0]:
print(score,acc)

1.7006362893364646 0.7024793309613693
